In [83]:
import pandas as pd
import yaml
from tabulate import tabulate

In [4]:
datasets = ['kuhar', 'motionsense', 'uci', 'wisdm', 'realworld_thigh', 'realworld_waist']
for dataset in datasets:
    with open(f'../execute_once_experiments/TVT_sb_no_reducer/scores/TVT_sb_no_reducer_{dataset}.yaml') as f:
        scores = yaml.load(f, Loader=yaml.FullLoader)
        print(dataset, scores['score'])

kuhar 0.8041666666666666
motionsense 0.8847457627118643
uci 0.8908695652173912
wisdm 0.7681972265023113
realworld_thigh 0.6959972394755003
realworld_waist 0.663233024691358


In [87]:
datasets = ['kuhar', 'motionsense', 'uci', 'wisdm', 'realworld_thigh', 'realworld_waist']
models = ['umap', 'ae', 'tae', 'convae', 'convtae']
percentages = [25, 50, 75, 100]

data = []
new_data = {
    'dataset': None,
    'model': None,
    'percentage': None,
    'score': None,
    'knn': None,
    'rf': None,
    'svm': None
}
for dataset in datasets:
    for model in models:
        for percentage in percentages:
            filename = f'../execute_once_experiments/TVT_sb_best_found_2023/scores/TVT_sb_{model}_{dataset}_P{percentage}.yaml'
            with open(filename) as f:
                scores = yaml.load(f, Loader=yaml.FullLoader)
                new_data['dataset'] = dataset
                new_data['model'] = model
                new_data['percentage'] = percentage
                new_data['score'] = scores['score']
                new_data['knn'] = scores['KNN-5-accuracy (mean)']
                new_data['rf'] = scores['randomforest-100-accuracy (mean)']
                new_data['svm'] = scores['SVM-rbf-C1.0-accuracy (mean)']
                data.append(new_data.copy())

In [92]:
data = pd.DataFrame(data)
# Drop rows with percentage 100 and 50
data = data[(data['percentage'] != 100) & (data['percentage'] != 50)]
# Add column with best string for each row
data['best'] = data[['knn', 'rf', 'svm']].idxmax(axis=1)
rows = []
for key, group in data.groupby(['model', 'percentage']):
    group.drop(['model', 'percentage', 'knn', 'rf', 'svm'], axis=1, inplace=True)
    # Round numbers
    
    group = group.set_index('dataset')
    group['score'] = group['score'].apply(lambda x: round(x*100, 1))
    group['VALUE'] = (group['score']).astype(str) + '%(' + group['best'].apply(str.upper) + ')'
    group = group.drop(['score', 'best'], axis=1)
    # print(group)
    group = group.T
    
    # print(group.loc['best'], '\nnnn')
    # Add key
    group['model'] = key[0]
    group['percentage'] = key[1]
    rows.append(group)
table3 = pd.concat(rows)
# Reorder columns
# print(tabulate(table3, headers='keys', tablefmt='psql'))
table3 = table3[['model', 'percentage', 'uci', 'motionsense', 'kuhar', 'wisdm', 'realworld_thigh', 'realworld_waist']]
# Change column names
table3 = table3.rename(columns={'uci': 'UCI', 'motionsense': 'MS', 'kuhar': 'KH', 'wisdm': 'WISDM', 'realworld_thigh': 'RW Thigh', 'realworld_waist': 'RW Waist'})
print(tabulate(table3, headers='keys', tablefmt='latex'))
# display(table3)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    


\begin{tabular}{llrllllll}
\hline
       & model   &   percentage & UCI        & MS         & KH         & WISDM      & RW Thigh   & RW Waist   \\
\hline
 VALUE & ae      &           25 & 72.8\%(SVM) & 79.5\%(KNN) & 66.7\%(SVM) & 70.2\%(KNN) & 65.8\%(SVM) & 60.6\%(SVM) \\
 VALUE & ae      &           75 & 73.6\%(SVM) & 79.7\%(KNN) & 68.1\%(SVM) & 70.7\%(RF)  & 68.9\%(RF)  & 60.5\%(SVM) \\
 VALUE & convae  &           25 & 20.0\%(KNN) & 79.8\%(KNN) & 63.9\%(KNN) & 66.0\%(RF)  & 61.1\%(KNN) & 68.4\%(SVM) \\
 VALUE & convae  &           75 & 68.6\%(KNN) & 73.7\%(KNN) & 66.0\%(KNN) & 20.0\%(KNN) & 72.7\%(KNN) & 16.7\%(RF)  \\
 VALUE & convtae &           25 & 74.8\%(RF)  & 73.7\%(RF)  & 59.7\%(SVM) & 65.8\%(RF)  & 66.8\%(RF)  & 67.3\%(SVM) \\
 VALUE & convtae &           75 & 63.3\%(RF)  & 60.2\%(RF)  & 63.1\%(RF)  & 69.6\%(RF)  & 48.0\%(RF)  & 58.9\%(RF)  \\
 VALUE & tae     &           25 & 70.5\%(RF)  & 75.2\%(RF)  & 65.3\%(SVM) & 67.1\%(RF)  & 62.8\%(RF)  & 65.9\%(SVM) \\
 VALUE & tae 